In [1]:
import os
import re
import pandas as pd
import numpy as np
from natsort import natsorted, index_natsorted, order_by_index
from tools import * # функции для обработки
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 1000)

### Обработка файлов формы № 10.3

In [2]:
year = 2020

Первый лист .xls-файла мы пропускаем, так как во всех отчетных формах там содержится общая информация, а не данные.

In [3]:
mailFile = 'ExcelFiles/Main/' + str(year) + 'main.xls'
mainDF = pd.read_excel(mailFile, 1, header=0)

Удаляем строку "Всего по составам УК РФ" значения сохраняем в отдельную переменную - они будут нужны позже, для проверки корректности обработки данных.

In [4]:
mainDF['clause'] = mainDF['clause'].astype('str')
total_values = list(mainDF[mainDF['clause'].str.contains("Всего по составам УК РФ|Всего лиц по составам УК РФ")].values[0][1:])
mainDF = mainDF[~mainDF['clause'].str.contains('Всего по составам УК РФ')]

In [5]:
mainDF = cleanClauseCol(mainDF) # чистим колонку с номерами статей от лишнего текста
mainDF = clauses2column(mainDF) # используя регулярное выражение, создаем колонки со номерами статей и частей 
firstCols = ['clause', 'part', 'totalConvicted']
mainDF = rearrangeCols(mainDF, firstCols) # меняем порядок столбцов для удобства

# поскольку названия частей вместе с примечаниями мы берем из формы 10-а, в тех случаях, где номера статей в формах 
# 10.3 и 10.3.1 и 10-а расходятся (в одной таблице - номер устаревшей статьи, а в друой - уже новой)
# - ориентируемся на 10-а

mainDF['part'] = mainDF['part'].replace('168$', '168ч.1', regex=True)

mainDF = sortTable(mainDF) # упорядочиваем строки

/home/kate/dostoevsky-data/tools.py:159: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clause'] = df['clause'].str.replace("Составы преступлений, введенные в УК РФ после утверждения форм отчетности:\nКлевета \(введ. ФЗ от 28.07.2012 N 141-ФЗ\) ст. ", "")
/home/kate/dostoevsky-data/tools.py:160: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clause'] = df['clause'].str.replace("Составы преступлений, введенные в раздел IX \"Преступления против общественной безопасности и общественного порядка\" УК РФ после утверждения форм отчетности приказом № 127\s+ст. ", "")
/home/kate/dostoevsky-data/tools.py:161: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clause'] = df['clause'].str.replace("Клевета в отношении судьи, присяжного заседателя, прокурора, следователя, лица, производящего дознание, судебного пристава \(введ. ФЗ от 28.07.2012 N

Выше мы сохранили значения строки "Всего по составам УК РФ", которая содержит сумммы по каждому столбцу. Теперь мы используем их для проверки корректности обработки данных, сравнив суммы по столбцам со значениями, указанными в строке "Всего по составам УК РФ".

К сожалению, как показала практика, числа эти не всегда совпадают, поэтому каждое несовпадение мы перепроверили вручную. Кроме того, еще в одном случае суммы действительно не совпадают из-за удаления в файлах 2011 год строки "Преступления, введенные в УК РФ после утверждения форм отчетности приказом № 115 от 10.06.2011" с одним осужденным к ограничению свободы (см. выше).

In [6]:
# выводит несоответствия сумм, если ничего не печатает - несоответствий нет
compareSums(mainDF, mainDF.columns, total_values, 2)

In [7]:
mainDF.head()

,clause,part,totalConvicted,primaryLifeSentence,primaryImprisonment,primarySuspended,primaryMilitaryDisciplinaryUnit,primaryArrest,primaryRestrain,primaryRestrictionsInMilitaryService,primaryCorrectionalLabour,primaryCommunityService,primaryForcedLabour,primaryDisqualification,primaryFine,primaryOther,exemptionTimeServedFromImprisonment,exemptionTimeServedOther,exemptionAmnestyFromImprisonment,exemptionAmnestyOther,exemptionOtherGroundsFromImprisonment,exemptionOtherGroundsOther,acquittal,dismissalAbsenceOfEvent,dismissalAmnesty,dismissalReconciliation,dismissalRepentance,dismissalCourtFine,dismissalOther,dismissalRepentance2,coerciveMeasures,addDisqualification,addFine,addTitlesWithdraw,addRestrain,unfinishedOffence,noCrimeSelf-defence,noCrimeNecessity,noCrimeOther
0,105,105ч.1,4027,0,3993,27,0,0,0,0,0,0,0,0,0,0,0,1,0,0,6,0,23,1,0,0,0,0,81,0,355,4,0,3,394,553,1,0,0
1,105,105ч.2,987,34,951,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,18,2,0,0,0,0,16,0,110,2,0,2,681,207,0,0,0
2,106,106,29,0,11,9,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,6,0,0,0
3,107,107ч.1,37,0,0,1,0,0,25,0,6,0,1,0,0,1,0,2,0,0,0,1,0,0,0,5,0,2,2,0,0,0,0,0,0,1,0,0,0
4,107,107ч.2,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


### Обработка файлов формы № 10.3.1

In [8]:
parametersFile = 'ExcelFiles/MainParameters/' + str(year) + 'parameters.xls'
parametersDF = pd.read_excel(parametersFile, 1, header=0)

parametersDF['clause'] = parametersDF['clause'].astype('str')
total_values = list(parametersDF[parametersDF['clause'].str.contains("Всего по составам УК РФ|Всего лиц по составам УК РФ")].values[0][1:])
parametersDF = parametersDF[~parametersDF['clause'].str.contains('Всего по составам УК РФ')]
parametersDF = cleanClauseCol(parametersDF)
parametersDF = clauses2column(parametersDF)
parametersDF['part'] = parametersDF['part'].replace('168$', '168ч.1', regex=True)
firstCols = ['clause', 'part']
parametersDF = rearrangeCols(parametersDF, firstCols)
parametersDF = sortTable(parametersDF)

/home/kate/dostoevsky-data/tools.py:159: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clause'] = df['clause'].str.replace("Составы преступлений, введенные в УК РФ после утверждения форм отчетности:\nКлевета \(введ. ФЗ от 28.07.2012 N 141-ФЗ\) ст. ", "")
/home/kate/dostoevsky-data/tools.py:160: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clause'] = df['clause'].str.replace("Составы преступлений, введенные в раздел IX \"Преступления против общественной безопасности и общественного порядка\" УК РФ после утверждения форм отчетности приказом № 127\s+ст. ", "")
/home/kate/dostoevsky-data/tools.py:161: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clause'] = df['clause'].str.replace("Клевета в отношении судьи, присяжного заседателя, прокурора, следователя, лица, производящего дознание, судебного пристава \(введ. ФЗ от 28.07.2012 N

In [9]:
parametersDF.head()

,clause,part,primaryImprisonment1,primaryImprisonment1_2,primaryImprisonment2_3,primaryImprisonment3_5,primaryImprisonment5_8,primaryImprisonment8_10,primaryImprisonment10_15,primaryImprisonment15_20,primaryImprisonmentUnderLowerLimit,primaryFine5,primaryFine5_25,primaryFine25_100,primaryFine100_300,primaryFine300_500,primaryFine500_1M,primaryFine1M,addFine5,addFine5_25,addFine25_100,addFine100_300,addFine300_500,addFine500_1M,addFine1M,dismissalCourtFine5,dismissalCourtFine5_25,dismissalCourtFine25_100,dismissalCourtFine100
0,105,105ч.1,1,2,34,153,1876,1515,412,0,150,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,105,105ч.2,1,1,2,18,71,156,393,309,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,106,106,2,8,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,107,107ч.1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
4,107,107ч.2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Снова проверяем суммы по столбцам. В двух столбцах они не сходятся: эта проблема есть и в исходных данных.

In [10]:
compareSums(parametersDF, parametersDF.columns, total_values, 2)


Сумма значений в колонке dismissalCourtFine5 17335 
 Значение в строке "Всего по составам УК РФ"  17336

Сумма значений в колонке dismissalCourtFine5_25 33155 
 Значение в строке "Всего по составам УК РФ"  33164


### Обработка файлов формы № 10-а 

In [11]:
addFile = 'ExcelFiles/Add/' + str(year) + 'add.xls'

columns = {
    'Число осужденных по основной статье': 'totalConvictedMain',
    'Виды преступлений': 'name',
    'Статьи Уголовного кодекса Российской Федерации по частям': 'clause',
    'Статьи УК РФ по частям': 'clause',
    'Дополнительная квалификация. Осуждено по числу лиц':  'addTotalPersons',
       'Дополнительная квалификация. Осуждено по количеству составов преступлений': 'addTotalOffences',
       'Дополнительная квалификация. Оправдано  по числу лиц': 'addAcquittalPersons',
       'Дополнительная квалификация. Оправдано  по количеству составов преступлений': 'addAcquittalOffences',
       'Дополнительная квалификация. Прекращено за отсутствием состава, события преступления, непричастностью к преступлению по числу лиц': 'addDismissalPersons',
       'Дополнительная квалификация. Прекращено за отсутствием состава, события преступления, непричастностью к преступлению по количеству составов преступлений': 'addDismissalOffences',
       'Дополнительная квалификация. Прекращено по иным основаниям по числу лиц': 'addDismissalOtherPersons',
       'Дополнительная квалификация. Прекращено по иным основаниям по количеству составов преступлений': 'addDismissalOtherOffences',
       'Принудительные меры медицинского характера в отношении невменяемого дополнительная квалификация по судебным постановлениям по количеству составов преступлений': 'addUnfitToPleadOffences',
    
}

df_list = pd.read_excel(addFile, sheet_name=None, header = None)
dfs = []
for key in list(df_list.keys())[2:-2]:
    df = df_list[key] # df_list[ 'Раздел 20 (Гл. 33)']
    df = dropFirstRows(df, 'Виды преступлений')
    df = dropFirstRows(df, '[а-яА-Я]{3,}', 1)
    df = deleteUnusedCols(df, year)
    df = tackleMergedCells(df)
    df.rename(columns=columns, inplace=True)
    df = df[df.columns[~df.columns.str.contains('[а-яА-Я]{2,}', regex=True)]]
    df.dropna(axis='rows', how='any', inplace=True)
    dfs.append(df)
addDF = pd.concat(dfs, ignore_index=True)
addDF['clause'] = addDF['clause'].astype('str')
addDF = addDF[~addDF['name'].str.contains('Виды преступлений')]

/home/kate/dostoevsky-data/tools.py:15: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, val in df.iloc[:,0].str.contains(value2search).iteritems():
/home/kate/dostoevsky-data/tools.py:67: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, col in df.iloc[0:header_rows].iteritems():
/home/kate/dostoevsky-data/tools.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[0:1].fillna(method='ffill', axis='columns', inplace=True)
/home/kate/dostoevsky-data/tools.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

In [12]:
addDF.head()

,name,clause,totalConvictedMain,addTotalPersons,addTotalOffences,addAcquittalPersons,addAcquittalOffences,addDismissalPersons,addDismissalOffences,addDismissalOtherPersons,addDismissalOtherOffences,addUnfitToPleadOffences
1,Убийство,105 ч. 1,4027,60,63,5,5,0,0,1,1,15
2,Убийство при отягчающих обстоятельствах,105 ч. 2,987,31,88,9,9,6,6,0,0,14
3,Убийство матерью новорожденного ребенка,106,29,0,0,0,0,1,1,0,0,0
4,"Убийство, совершенное в состоянии аффекта",107 ч. 1,37,3,3,0,0,0,0,0,0,0
5,Убийство в состоянии аффекта двух или более лиц,107 ч. 2,1,0,0,0,0,0,0,0,0,0


In [13]:
addDF = cleanClauseCol(addDF)
addDF = clauses2column(addDF)

firstCols = ['name', 'clause', 'part', 'totalConvictedMain']
addDF = rearrangeCols(addDF, firstCols)

addDF['part'] = addDF['part'].replace('136', '136ч.2')
addDF = addMilitaryOfences(addDF)
addDF['name'] = addDF['name'].str.replace('^\s{1}', '')
addDF['name'] = addDF['name'].str.replace('\n', '') 
addDF = sortTable(addDF)

/home/kate/dostoevsky-data/tools.py:159: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clause'] = df['clause'].str.replace("Составы преступлений, введенные в УК РФ после утверждения форм отчетности:\nКлевета \(введ. ФЗ от 28.07.2012 N 141-ФЗ\) ст. ", "")
/home/kate/dostoevsky-data/tools.py:160: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clause'] = df['clause'].str.replace("Составы преступлений, введенные в раздел IX \"Преступления против общественной безопасности и общественного порядка\" УК РФ после утверждения форм отчетности приказом № 127\s+ст. ", "")
/home/kate/dostoevsky-data/tools.py:161: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clause'] = df['clause'].str.replace("Клевета в отношении судьи, присяжного заседателя, прокурора, следователя, лица, производящего дознание, судебного пристава \(введ. ФЗ от 28.07.2012 N

<a id='data_check'></a>

### Проверка корректности обработки данных

Теперь нужно соединить все три таблицы за год в одну, но сначала проведем несколько проверок.

1) все три таблицы должны быть одной длины.

In [14]:
checkTablesLen(mainDF, addDF, parametersDF)
checkTablesLen(mainDF, addDF)

ОК: длины таблиц № 10.3 и № 10.3.1 совпадают
ОК: длины таблиц № 10.3 и № 10-а совпадают
ОК: длины таблиц № 10.3 и № 10-а совпадают


2) номера статей при построчном сравнении должны совпадать. Однако простое построчное сравнение может нас обмануть.В отчетных формах Судебного департамента могут соседствовать актуальная и утратившая силу части одной и той же статьи с одинаковыми номерами, при этом все примечания к номерам статей мы убрали, оставив их у названий статей/частей. Поэтому построчно имеет смысл сравнивать не только колонку с номерами статей, но и числа в тех столбцах, которые в разных формах должны содержать одинаковую информацию.


Так в форме № 10.3 и № 10.3.1 должны совпадать соответственно колонки "Лишение свободы" и сумма чисел из колонок "До 1 года вкл.", "Свыше 1 до 2 лет вкл.",	"Свыше 2 до 3 лет вкл.", "Свыше 3 до 5 лет вкл.", "Свыше 5 до 8 лет вкл.", "Свыше 8 до 10 лет вкл.", "Свыше 10 до 15 лет вкл.", "Свыше 15 до 20 лет вкл.".

А в формах № 10.3 и № 10-а - колонки "ВСЕГО ОСУЖДЕНО ЛИЦ" и "Число осужденных лиц по основной статье".

На практике так происходит не всегда и числа регулярно расходятся, поэтому приходится ориентироваться не на их полное совпадение, а на величину разницы между ними.

In [15]:
checkNumbersBetweenForms(year, mainDF, addDF, parametersDF)

Не совпадает число осужденных по основной статье. 
Статья:  Воинские преступления 
10.3: 979 
10-a: nan 
Год: 2020


Теперь можно свести все таблицы за один год в одну, удалив дублирующиеся столбцы, а именно столбцы с номером и частью статьи в формах № 10.3.1 и № 10-а и столбцы с числом осужденных по основному составу преступления в форме № 10-а.

In [16]:
addDF = addDF.drop(['clause', 'part', 'totalConvictedMain'], axis=1)
parametersDF = parametersDF.drop(['clause', 'part'], axis=1)
df = pd.concat([mainDF, parametersDF, addDF], axis=1)

firstCols = ['name', 'clause', 'part', 'totalConvicted']
df = rearrangeCols(df, firstCols)

In [17]:
df.head()

,name,clause,part,totalConvicted,primaryLifeSentence,primaryImprisonment,primarySuspended,primaryMilitaryDisciplinaryUnit,primaryArrest,primaryRestrain,primaryRestrictionsInMilitaryService,primaryCorrectionalLabour,primaryCommunityService,primaryForcedLabour,primaryDisqualification,primaryFine,primaryOther,exemptionTimeServedFromImprisonment,exemptionTimeServedOther,exemptionAmnestyFromImprisonment,exemptionAmnestyOther,exemptionOtherGroundsFromImprisonment,exemptionOtherGroundsOther,acquittal,dismissalAbsenceOfEvent,...,primaryFine25_100,primaryFine100_300,primaryFine300_500,primaryFine500_1M,primaryFine1M,addFine5,addFine5_25,addFine25_100,addFine100_300,addFine300_500,addFine500_1M,addFine1M,dismissalCourtFine5,dismissalCourtFine5_25,dismissalCourtFine25_100,dismissalCourtFine100,addTotalPersons,addTotalOffences,addAcquittalPersons,addAcquittalOffences,addDismissalPersons,addDismissalOffences,addDismissalOtherPersons,addDismissalOtherOffences,addUnfitToPleadOffences
0,Убийство,105,105ч.1,4027,0,3993,27,0,0,0,0,0,0,0,0,0,0,0,1,0,0,6,0,23,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,60,63,5,5,0,0,1,1,15
1,Убийство при отягчающих обстоятельствах,105,105ч.2,987,34,951,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,18,2,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,31,88,9,9,6,6,0,0,14
2,Убийство матерью новорожденного ребенка,106,106,29,0,11,9,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0
3,"Убийство, совершенное в состоянии аффекта",107,107ч.1,37,0,0,1,0,0,25,0,6,0,1,0,0,1,0,2,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,3,3,0,0,0,0,0,0,0
4,Убийство в состоянии аффекта двух или более лиц,107,107ч.2,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


И привести таблицу в длинную форму.

In [18]:
dfMelted = meltTable(df, year)

In [19]:
dfMelted.head(20)

,year,clause,part,name,parameter,value
0,2020,105,105ч.1,Убийство,totalConvicted,4027
1,2020,105,105ч.2,Убийство при отягчающих обстоятельствах,totalConvicted,987
2,2020,106,106,Убийство матерью новорожденного ребенка,totalConvicted,29
3,2020,107,107ч.1,"Убийство, совершенное в состоянии аффекта",totalConvicted,37
4,2020,107,107ч.2,Убийство в состоянии аффекта двух или более лиц,totalConvicted,1
5,2020,108,108ч.1,Убийство при превышении пределов необходимой о...,totalConvicted,227
6,2020,108,108ч.2,"Убийство при превышении мер, необходимых для з...",totalConvicted,0
7,2020,109,109ч.1,Причинение смерти по неосторожности,totalConvicted,772
8,2020,109,109ч.2,Причинение смерти по неосторожности при отягча...,totalConvicted,114
9,2020,109,109ч.3,Причинение смерти по неосторожности двум или б...,totalConvicted,39


In [20]:
dfMelted.to_pickle('2020.pkl')